<h1 style="color:rgb(0,120,170)">344.038, KV Multimedia Search and Retrieval (WS2023/24)</h1>
<h2 style="color:rgb(0,120,170)">Task 2_Group B</h2>

| First Name | Family Name  | Matr.Nr   |
|:-----------|:-------------|:----------|
| Branko     | Paunović     | K12046370 |
| Lukas      | Troyer       | K12006666 |
| Hadi       | Sanaei       | K11733444 |

# Retrieval systems

#### Import datasets, setup helper interfaces

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.decomposition import PCA

from song import songs
from datasets import datasets, LocalDataset
from retrieval import Retrieval

#### Prompt for user input

In [ ]:
# Number of similar songs to retrieve
num_top_similar = int(input("Enter the number of top similar tracks: "))

# Example:
#   - Title: Letterman
#   - Artist: Wiz Khalifa
query, query_row_id = songs.prompt_song_query()

## Video-Based(\<similarity\>, \<feature\>)
### 1. Video-based(similarity, incp)

In [ ]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.resnet))

## Fusion-Based (\<similarity\>, \<feature\>)
### Early Fusion

In [ ]:
def early_fusion_dataframe(d1: LocalDataset, d2: LocalDataset):
    if len(d1.df) != len(d2.df):
        raise ValueError("Both dataframes must have the same number of rows")

    # Concatenate dataframes along columns (axis=1)
    df1_pca = pd.DataFrame(PCA(n_components=0.8).fit_transform(d1.df.iloc[:, 1:]))
    df1_pca["id"] = d1.df["id"]

    df2_pca = pd.DataFrame(PCA(n_components=0.8).fit_transform(d2.df.iloc[:, 1:]))
    df2_pca["id"] = d2.df["id"]
    
    fused_df = pd.concat([df1_pca, df2_pca], axis=1)  
    return fused_df


df_ef = early_fusion_dataframe(datasets.musicnn, datasets.resnet)
dataset_early_fusion = LocalDataset("early_fusion-musicnn_resnet", df_ef)

In [ ]:
df_ef.head()

In [ ]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=dataset_early_fusion))

### Late Fusion

In [ ]:
#
# Late fusion - Score Aggregation Fusion for two DataFrames (text and audio)
#
def late_fusion_df(d1: LocalDataset, d2: LocalDataset, d1_weight: float, d2_weight: float):
    # Rename the 'similarity' column
    df_1 = d1.df.copy().rename(columns={'similarity': 'similarity_d1'})
    df_2 = d2.df.copy().rename(columns={'similarity': 'similarity_d2'})

    # Merge DataFrames on 'id'
    merged_df = pd.merge(df_2[['id', 'similarity_d2']], df_1, on='id')

    # Assuming the scores are already normalized between 0 and 1
    normalized_d1_scores = merged_df['similarity_d1'].values
    normalized_d2_scores = merged_df['similarity_d2'].values

    # Aggregate scores
    aggregated_scores = d1_weight * normalized_d1_scores + d2_weight * normalized_d2_scores

    # Add the aggregated scores to the DataFrame
    merged_df['aggregated_score'] = aggregated_scores

    # Sort DataFrame based on aggregated scores
    sorted_fusion_df = merged_df.sort_values(by='aggregated_score', ascending=False)
    return sorted_fusion_df

fusion_results_df = late_fusion_df(
    datasets.tf_idf,
    datasets.musicnn,
    d1_weight=0.6,
    d2_weight=0.4
)
fusion_results_df

In [ ]:
#
# Late fusion - Rank-Level Fusion for two DataFrames (video and audio)
#
def rank_level_fusion_video_audio(d1: LocalDataset, d2: LocalDataset, d1_weight: float, d2_weight: float):
    # Rename the 'similarity' column
    video_scores = d1.df.copy().rename(columns={'similarity': 'similarity_video'})
    audio_scores = d2.df.copy().rename(columns={'similarity': 'similarity_audio'})

    # Merge DataFrames on 'id'
    merged_df = pd.merge(audio_scores[['id', 'similarity_audio']], video_scores, on='id')

     # Assuming the scores are already normalized between 0 and 1
    normalized_video_scores = merged_df['similarity_video'].values
    normalized_audio_scores = merged_df['similarity_audio'].values

    # Convert similarity scores to ranks using pandas Series
    video_ranks = pd.Series(normalized_video_scores).rank(ascending=False)
    audio_ranks = pd.Series(normalized_audio_scores).rank(ascending=False)

    # Rank-level fusion: Combine the ranks using weighted sum
    aggregated_ranks = d1_weight * video_ranks + d2_weight * audio_ranks

    # Add the aggregated ranks to the DataFrame
    merged_df['aggregated_rank'] = aggregated_ranks

    # Sort DataFrame based on aggregated ranks
    sorted_fusion_df = merged_df.sort_values(by='aggregated_rank')
    return sorted_fusion_df

# Example usage
fusion_results_df = rank_level_fusion_video_audio(
    datasets.resnet,
    datasets.musicnn,
    d1_weight=0.6,
    d2_weight=0.4
)


In [ ]:
# Display the fused results
print("Rank-Level Fusion Results:")
print(fusion_results_df[['id', 'similarity_video', 'similarity_audio', 'aggregated_rank']])

# Evaluation

In [ ]:
# Obtain necessary information and store for easy interop
from genres import Genres

genres = Genres()

## Accuracy
#### 1. Precision@k & Recall@k

In [ ]:
from precision_recall import PrecisionRecall

pr = PrecisionRecall(genres)
pr.compute()

In [ ]:
pr.plot_each()

In [ ]:
pr.plot_all_single()

#### 2. nDCG@10

In [ ]:
from ndcg import Ndcg

ndcg = Ndcg(genres)
ndcg.compute()
ndcg.plot()

## Beyond Accuracy
### 1. Genre Coverage@10

In [ ]:
from genre_coverage import GenreCoverage

genres_coverage = GenreCoverage(genres)
genres_coverage.compute()
genres_coverage.plot()

### 2. Genre Diversity@10

In [ ]:
from genre_diversity import GenreDiversity

genres_diversity = GenreDiversity(genres)
genres_diversity.compute()
genres_diversity.plot()

# Frontend

In [ ]:
# precompute retrievals (for frontend and chached lookup)

# only do this if you want to recompute the retrieval

#retrieval = Retrieval(n=100)
#retrieval.precompute_all(threads=4)

In [ ]:
from utils import write_song_df_to_json_file

write_song_df_to_json_file("frontend/static/songMeta.json", datasets.information.df, datasets.url.df, datasets.genres.df)

# frontend uses frontend/static/songMeta.json and retrievals/*.json
